In [5]:
import pygame
import sys

pygame.init()

# 화면 크기 설정
width, height = 800, 600
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption("라면 게임")

# 색상 및 폰트 설정
YELLOW = (255, 255, 0)
BLUE = (0, 0, 255)
RED = (255, 0, 0)
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GRAY = (200, 200, 200)

font_path = "C:/Windows/Fonts/malgun.ttf"
font = pygame.font.Font(font_path, 40)
small_font = pygame.font.Font(font_path, 30)

# 배경 이미지 설정
background_img = pygame.image.load("img.png")
background_img = pygame.transform.scale(background_img, (width, height))

# 점수 초기화
score = 0

# 주전자 이미지 및 위치 설정
kettle_img = pygame.image.load("kettle.png")
kettle_img = pygame.transform.scale(kettle_img, (200, 200))  # 이미지 크게 조정
kettle_pos = [20, height - 120]  # 이미지 위치
kettle_dragging = False  # 드래그 여부 플래그

# 라면을 제출하는 버튼
class Button:
    def __init__(self, x, y, width, height, text, action=None, color=(255, 232, 73)):
        self.rect = pygame.Rect(x, y, width, height)
        self.text = text
        self.action = action
        self.color = color

    def draw(self, screen):
        pygame.draw.rect(screen, self.color, self.rect)
        text_surface = font.render(self.text, True, BLACK)
        screen.blit(text_surface, (self.rect.centerx - text_surface.get_width() // 2, self.rect.centery - text_surface.get_height() // 2))

    def is_clicked(self, pos):
        return self.rect.collidepoint(pos)

# 라면 상태를 처리하는 클래스
class Block:
    def __init__(self, color, x, y, size=50):
        self.color = color
        self.rect = pygame.Rect(x, y, size, size)
        self.dragging = False

    def draw(self, screen):
        pygame.draw.rect(screen, self.color, self.rect)

    def handle_event(self, event):
        if event.type == pygame.MOUSEBUTTONDOWN and self.rect.collidepoint(event.pos):
            self.dragging = True
        elif event.type == pygame.MOUSEBUTTONUP:
            self.dragging = False
        elif event.type == pygame.MOUSEMOTION and self.dragging:
            self.rect.move_ip(event.rel)

def show_score():
    global score
    score_text = font.render(f"점수: {score}원", True, BLACK)
    screen.blit(score_text, (20, 20))

# 라면 제출 후 평가하는 함수
def evaluate_ramen(cooking_time, correct_order):
    global score
    if not correct_order:
        score += 2000
    elif cooking_time >= 10:
        score += 0  # 불탐 상태로 실패
    elif cooking_time >= 4:
        score += 3000  # 적당한 상태
    elif cooking_time >= 2:
        score += 1000  # 미약 상태
    return True  # 라면 제출 후 게임 상태 초기화

def start_game():
    global score, kettle_dragging, kettle_pos

    yellow_block = Block(YELLOW, 100, 100)
    blue_block = Block(BLUE, 300, 100)
    red_block = Block(RED, 500, 100)

    submit_button = Button(650, 500, 120, 50, "라면 제출", action=None)

    start_ticks = pygame.time.get_ticks()
    total_time = 60
    game_over = False
    cooking_time = 0
    correct_order = True

    while True:
        elapsed_time = (pygame.time.get_ticks() - start_ticks) / 1000
        remaining_time = max(0, total_time - elapsed_time)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()

            # 주전자 드래그 처리
            if event.type == pygame.MOUSEBUTTONDOWN:
                if kettle_img.get_rect(topleft=kettle_pos).collidepoint(event.pos):
                    kettle_dragging = True
                    mouse_x, mouse_y = event.pos
                    offset_x = kettle_pos[0] - mouse_x
                    offset_y = kettle_pos[1] - mouse_y

                if submit_button.is_clicked(event.pos):
                    evaluate_ramen(cooking_time, correct_order)
                    yellow_block = Block(YELLOW, 100, 100)
                    blue_block = Block(BLUE, 300, 100)
                    red_block = Block(RED, 500, 100)
                    cooking_time = 0

            elif event.type == pygame.MOUSEBUTTONUP:
                kettle_dragging = False

            elif event.type == pygame.MOUSEMOTION and kettle_dragging:
                mouse_x, mouse_y = event.pos
                kettle_pos[0] = mouse_x + offset_x
                kettle_pos[1] = mouse_y + offset_y

            if not game_over:
                yellow_block.handle_event(event)
                blue_block.handle_event(event)
                red_block.handle_event(event)

        screen.fill(WHITE)

        if not game_over:
            yellow_block.draw(screen)
            blue_block.draw(screen)
            red_block.draw(screen)
            submit_button.draw(screen)

            show_score()

            # 주전자 이미지 표시 (위치 반영)
            screen.blit(kettle_img, kettle_pos)

        # 타이머 바
        bar_width = 300
        bar_height = 30
        time_ratio = remaining_time / total_time
        pygame.draw.rect(screen, GRAY, (250, 20, bar_width, bar_height))
        pygame.draw.rect(screen, (233, 55, 113), (250, 20, int(bar_width * time_ratio), bar_height))

        if remaining_time <= 0 and not game_over:
            game_over = True

        if game_over:
            end_text = font.render("게임 종료!", True, RED)
            screen.blit(end_text, (width // 2 - end_text.get_width() // 2,
                                   height // 2 - end_text.get_height() // 2))

        pygame.display.update()

        if game_over:
            pygame.time.wait(3000)
            return

def show_instructions():
    instructions = [
        "게임 설명:",
        "색깔 블록을 드래그하여 재료를 넣고,",
        "라면을 제출 버튼을 눌러 평가 받으세요!",
        "시간이 다 되기 전에 최대한 많은 라면을 만드세요!"
    ]
    for idx, line in enumerate(instructions):
        text_surface = small_font.render(line, True, BLACK)
        screen.blit(text_surface, (100, 150 + idx * 40))

game_button = Button(300, 500, 200, 50, "게임으로", action=start_game)

def loading_screen():
    start_button = Button(300, 250, 200, 50, "Start", start_game, color=(255, 232, 73))
    instructions_button = Button(300, 350, 200, 50, "게임 설명", action=None, color=(255, 232, 73))
    
    show_instructions_flag = False

    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.MOUSEBUTTONDOWN:
                pos = event.pos
                if not show_instructions_flag:
                    if start_button.is_clicked(pos):
                        start_button.action()
                    elif instructions_button.is_clicked(pos):
                        show_instructions_flag = True
                else:
                    if game_button.is_clicked(pos):
                        game_button.action()

        screen.blit(background_img, (0, 0))
        if not show_instructions_flag:
            start_button.draw(screen)
            instructions_button.draw(screen)
        else:
            show_instructions()
            game_button.draw(screen)

        pygame.display.update()

    pygame.quit()
    sys.exit()

# 실행
loading_screen()


SystemExit: 